In [1]:
import torch

In [32]:
# https://github.com/pytorch/pytorch/issues/47841
def torch_binom(n, v):
    #mask = n.detach() >= v.detach()
    #n = mask * n
    #v = mask * v
    a = torch.lgamma(n + 1) - torch.lgamma((n - v) + 1) - torch.lgamma(v + 1)
    return torch.exp(a)# * mask

In [33]:
# https://en.wikipedia.org/wiki/Bernstein_polynomial
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.BPoly.html
def b(v, n, x):
    return torch_binom(n, v) * x**v * (1 - x)**(n - v)

In [34]:
b(v = torch.tensor([2]),
  n = torch.tensor([5]),
  x = torch.tensor([10]))

tensor([-729000.])

In [39]:
n = torch.tensor([5])
x = torch.tensor([10])
params = torch.tensor([1,1,1,1,1])

prediction = sum(params * b(torch.tensor([v]), n, x) for v in range(n+1))

prediction

tensor([1., 1., 1., 1., 1.])